[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/multi-modal/recipe-search/recipe-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/multi-modal/recipe-search/recipe-search.ipynb)

<h1>Inverse recipe search</h1>

This notebook demonstrates how Pinecone can handle very large embeddings that come from a sophisticated model trained using a large multimodal food dataset. The paper is available at this link: https://research.fb.com/wp-content/uploads/2019/05/Inverse-Cooking-Recipe-Generation-from-Food-Images.pdf.

We have built this recipe search by image using Pinecone in the following way:
<br>
- From Facebook's large model, we have extracted 512 x 7 x 7 large embeddings from their convolutional network
<br>
- We have used these features to obtain ingredients and instructions vocabularies
<br>
- We have inserted these large embeddings into Pinecone
<br>
- When querying, each image is passed through the network and a large embedding is obtained
<br>
- We used this large embedding to find the most similar embeddings
<br>
- When similar images are found, we checked our vocabularies for ingredients and instructions
<br>



Recipe generation model extracts image features e<sub>I</sub> with the image encoder, parametrized by θ<sub>I</sub> . Ingredients are predicted by θ<sub>L</sub>, and encoded into ingredient embeddings e<sub>L</sub> with θ<sub>e</sub>. The cooking instruction decoder, parametrized by θ<sub>R</sub> generates a recipe title and a sequence of cooking steps by attending to image embeddings e<sub>I</sub> , ingredient embeddings e<sub>L</sub> , and previously predicted words (r<sub>0</sub>, ..., r<sub>t−1</sub>).

<img src="images/facebook_model.png" />

Besides showcasing Facebook's solution, we have also checked what happens if we use much smaller embeddings (~10 times smaller) obtained from the same convolutional network (ResNet101) that was pre-trained on the imagenet dataset, and the same vocabularies to search for ingredients and instructions. 

We do this by using Facebook's instructions and ingredients vocabularies with a modified image encoder. 

Let's get started!

# Facebook's inversecooking model

## Dependencies

Install and import relevant python packages.

In [ ]:
!pip install matplotlib
!pip install torch
!pip install numpy
!pip install Pillow
!pip install tensorflow
!pip install gdown
!pip install pandas

We will now download already created ingredients and instructions vocabularies

In [ ]:
# ingredients vocabulary
!wget https://dl.fbaipublicfiles.com/inversecooking/ingr_vocab.pkl

In [ ]:
# instructions vocabulary
!wget https://dl.fbaipublicfiles.com/inversecooking/instr_vocab.pkl

Downloading Facebook's pretrained model

In [ ]:
!wget https://dl.fbaipublicfiles.com/inversecooking/modelbest.ckpt

We wil now clone Facebook's `inversecooking` repository. We are doing this because in the following steps we will be using some of the methods to access Facebook's model.

In [ ]:
!git clone https://github.com/facebookresearch/inversecooking.git

In [6]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import os
from inversecooking.src.args import get_parser
import pickle
from torchvision import transforms
from PIL import Image
import time
import requests
from io import BytesIO
import random
from collections import Counter

If you are planning on creating your own vectors, you should download a dataset from this link [Recipe1M](http://im2recipe.csail.mit.edu/dataset/download) (registration required). We have used the test set to demonstrate Pinecone's capabilities to handle large embeddings. Note that vector creation is a long process and should take around 10 hours for 10K vectors.

On the other hand, if you would like to use already created vectors, in the next step we will download them.

In [ ]:
!gdown --id 1gE258b6bvIMiM7hk0T96R5Fpt5XNHkAf  # downloading ingredients
!gdown --id 1tuTRqLrEU8cuUeQEx1TSFfFmJOfCLfYa  # downloading instructions
!gdown --id 1ep62ohbWP6Ywk7WPf7-mlKSShuZpp-cT  # downloading vectors generated by Facebook's model
!gdown --id 1PKzN2zdVBq5dlmDu2xACIUO3B6FAcS2_  # downloading vectors generated by a modified ResNet101

Specify the directory to which you will copy your downloaded images. Feel free to leave images within their subdirectories. The following code will fetch them properly.


In [8]:
data_dir = 'data'

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
map_loc = None if torch.cuda.is_available() else 'cpu'

Load vocabularies

In [10]:
ingrs_vocab = pickle.load(open('ingr_vocab.pkl', 'rb'))
vocab = pickle.load(open('instr_vocab.pkl', 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [11]:
print (instrs_vocab_size, ingr_vocab_size)

23231 1488


Load the model

In [ ]:
import sys; sys.argv=['']; sys.path.insert(1, 'inversecooking/src')
from inversecooking.src.model import get_model

args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False
model = get_model(args, ingr_vocab_size, instrs_vocab_size)

model_path = 'modelbest.ckpt'
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False

In [13]:
transf_list_batch = []
transf_list_batch.append(transforms.ToTensor())
transf_list_batch.append(transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225)))
to_input_transf = transforms.Compose(transf_list_batch)

## Preparing data

Now that we have loaded Facebook's pretrained model as well as ingredients and instructions vocabularies, we are ready to start creating embeddings.

You can choose to create image vectors or load prepared vectors. 
Setting `create_vectors` to `True` will, in the following steps, trigger vector creation from scratch. This is a long operation. Depending on the number of vectors you choose to create and whether you choose to use cpu or gpu, creating vectors can take around 10 hours.
Setting `create_vectors` to `False` will load vectors that have already been created.

In [15]:
create_vectors = False

Create a list of filepaths to all images in the dataset.

In [16]:
if create_vectors:
    show_anyways = False
    image_folder = data_dir

    demo_files = []
    for path, subdirs, files in os.walk(image_folder):
        for name in files:
            if name.endswith('.jpg'):
                demo_files.append(os.path.join(path, name))

We will now prepare vectors for upload.

In [17]:
if create_vectors:
    items_to_upload = []
    ingredients = {}
    instructions = {}
    
    i=0
    demo_files = demo_files[:10000]

    for img_file in demo_files:
        print(f'{i}/{len(demo_files)}', end='\r')
        i = i+1

        image = Image.open(img_file).convert('RGB')

        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

        img_features = model.image_encoder(image_tensor)
        features = model.image_encoder.linear(model.image_encoder.resnet(image_tensor))
        
        with torch.no_grad():
            outputs = model.sample(image_tensor, 
                                   greedy=True, 
                                   temperature=1.0, 
                                   beam=-1, 
                                   true_ingrs=None)

        ingr_ids = outputs['ingr_ids'].cpu().numpy()
        recipe_ids = outputs['recipe_ids'].cpu().numpy()

        items_to_upload.append((f'{img_file}', img_features.flatten().tolist()))

        ingredients[img_file] = ingr_ids[0]
        instructions[img_file] = recipe_ids[0]

In case you have chosen to create vectors from scratch, this step will save them so you don't have to recreate them.

In [18]:
if create_vectors:
    with open('vectors.pkl', 'wb') as f:
        pickle.dump(items_to_upload, f)
    with open('ingredients.pkl', 'wb') as f:
        pickle.dump(ingredients, f)
    with open('instructions.pkl', 'wb') as f:
        pickle.dump(instructions, f)

Load already created vectors.

In [19]:
if not create_vectors:
    items_to_upload = pickle.load(open('vectors.pkl', 'rb'))
    ingredients = pickle.load(open('ingredients.pkl', 'rb'))
    instructions = pickle.load(open('instructions.pkl', 'rb'))

Check vector size. This is the number of features in each vector.

In [20]:
len(items_to_upload[0][1])

25088

<h1>Pinecone</h1>

Install Pinecone client

In [ ]:
!pip install --quiet -U pinecone-client

Set up Pinecone. Get your Pinecone API key here: <a>https://www.pinecone.io/start/</a>.

In [29]:
import pinecone

# Load Pinecone API key
api_key = '<YOUR_API_KEY>'
pinecone.init(api_key=api_key)

In [30]:
index_name = 'recipe-search'

Check whether the index with the same name already exists

In [31]:
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

  0%|          | 0/1 [00:00<?, ?it/s]

Now that we have our vector embeddings we can create our Pinecone service and upload the data to it.

Below we create a Pinecone index. Here we use two arguments:

metric="cosine" means the vector search will use cosine as the measure of similarity.
shards=3 means the service will run on three nodes. If you have more vectors, you should increase the number of shards. Refer to the documentation for guidelines on how to choose the number of shards.

In [ ]:
pinecone.create_index(name=index_name, metric='cosine', shards=3)

Now that the index is created, we can upload vectorized images. To do that, we connect to the index. You only need to do this once.

In [33]:
index = pinecone.Index(name=index_name)
index.info()

InfoResult(index_size=0, dimension=0)

Upsert will upload, add, and index items into your remote vector similarity search service. If an item with that ID already exists it will be overwritten with the new provided value.

Upsert is a long operation. It should be complete in 1-2 hours.

<img src='images/upsert.png' />

## Upsert

In [ ]:
for i in range(0, len(items_to_upload), 100): 
    acks = index.upsert(items=items_to_upload[i:i+100])

In [35]:
index.info()

InfoResult(index_size=10000, dimension=0)

## Query

Now we will query for recipes for a number of food images from the internet.

We are only looking for similar images in Pinecone's index and later check which ingredients and instructions are specified for each resulting image. 

<img src='images/query.png' />

In [36]:
# add step for ingredients and instructions
demo_urls = [
            'https://a7m3f5i5.rocketcdn.me/wp-content/uploads/2015/09/moms-spaghetti-sauce-recipe-a-healthy-slice-of-life-6-of-6-800x600.jpg',
            'https://www.simplyrecipes.com/thmb/6azShcxiFtG0LlqXqz6eq3BM56A=/450x0/filters:no_upscale():max_bytes(150000):strip_icc()/__opt__aboutcom__coeus__resources__content_migration__simply_recipes__uploads__2007__03__spinach-horiz-640-2dcea783044340479666095cb40ab205.jpg',
            'https://sugargeekshow.com/wp-content/uploads/2020/10/baked_donut_recipe_featured.jpg',
            'https://www.moulinex-me.com/medias/?context=bWFzdGVyfHJvb3R8MTQzNTExfGltYWdlL2pwZWd8aDM2L2g1Mi8xMzA5NzI3MzI2MjExMC5qcGd8N2MxZDhmNmI5ZTgzZDZlZWQyZGQ4YjFlZjUyNDlkMTExYjdkZDdlZmFkY2I0N2NmNjljOGViNmExZjIyMDU4Yw',
            'https://hips.hearstapps.com/hmg-prod/images/delish-how-to-make-a-smoothie-horizontal-1542310071.png',
            'https://www.simplyrecipes.com/thmb/45-YxKaW3K8CYhYzfxnYUDhz63Q=/1800x1012/smart/filters:no_upscale()/__opt__aboutcom__coeus__resources__content_migration__simply_recipes__uploads__2007__01__roast-chicken-carrots-sally-horiz-a-1800-51888dd5b8c44b6e9e656092f4f96f12.jpg',
            'https://www.jessicagavin.com/wp-content/uploads/2018/02/greek-salad-2-1200.jpg',
            'https://www.twopeasandtheirpod.com/wp-content/uploads/2021/06/Greek-Quinoa-Salad4637.jpg',
            'https://www.onceuponachef.com/images/2020/02/Double-Chocolate-Muffins.jpg',
            'https://hips.hearstapps.com/del.h-cdn.co/assets/17/31/1501791674-delish-chicken-curry-horizontal.jpg?crop=1.00xw:0.750xh;0,0.159xh&resize=640:*'
            ]

In [ ]:
from inversecooking.src.utils.output_utils import prepare_output
from PIL import Image


titles_display = []
ingredients_display = []
instructions_display = []
images_display = []


thresh_score = 0.4

for img_file in demo_urls:
    response = requests.get(img_file)
    print("got response")
    image = Image.open(BytesIO(response.content))

    transf_list = []
    transf_list.append(transforms.Resize(256))
    transf_list.append(transforms.CenterCrop(224))
    transform = transforms.Compose(transf_list)
    print("got transform")

    image_transf = transform(image)
    image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)
    print("got tensor")

    img_features = model.image_encoder(image_tensor)
    sample_embedding = img_features.flatten().tolist()
    print("Got embedding")
    
    query_results = index.query(queries=[sample_embedding], top_k=2, disable_progress_bar=True)
    print("got results")
    
    for image_path, score in zip(query_results[0].ids, query_results[0].scores):
        if score > thresh_score:
            image = Image.open(image_path).convert('RGB')

            transf_list = []
            transf_list.append(transforms.Resize(256))
            transf_list.append(transforms.CenterCrop(224))
            transform = transforms.Compose(transf_list)

            image_transf = transform(image)

            images_display.append(image_path)
        
            outs, valid = prepare_output(instructions[img_file],
                                         ingredients[img_file], 
                                         ingrs_vocab, 
                                         vocab)

            if not valid:
                print ("Not a valid recipe!")
                print ("Reason: ", valid['reason'])
                continue
            else:
                titles_display.append(outs['title'])
                ingredients_display.append(', '.join(outs['ingrs']))
                instructions_display.append('-'+'\n-'.join(outs['recipe']))

        else:
            print('No recipes found.')
            break

## Display results

In [89]:
import requests
from IPython.display import Image, display
from IPython.core.display import HTML 


def path_to_image_html(path):
     return '<img src="'+ path + '" width="200"/>'


In [90]:
from IPython.display import display_html
from itertools import chain,cycle
import pandas as pd

def display_table(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:left"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html(escape=False, 
                             formatters=dict(query_image=path_to_image_html, 
                                             recipe_image=path_to_image_html)
                            ).replace('table','table style="display:inline;" width="100%"')
        html_str+='</td></th>'
    html_str = html_str.replace('<td>', '<td text-align="left">')
    html_str = html_str.replace('<th>', '<th text-align="left">')
    display_html(html_str,raw=True)

    


# Results
instructions_display = [instruction.replace('\n', '<br>') for instruction in instructions_display]

df_r = pd.DataFrame({'query_image': [entry for entry in demo_urls for _ in range(2)],
                     'similar recipe': titles_display, 
                     'recipe_image': images_display,
                     'ingredients': ingredients_display, 
                     'instructions': instructions_display})

# Show results
display_table(df_r, titles=['Results'])



,query_image,similar recipe,recipe_image,ingredients,instructions
0,,Spaghetti sauce,,"onion, pepper, tomato, clove, oil, salt, beef, oregano, basil, parsley","-In a large skillet, heat oil over medium heat.-Add onion and garlic and saute until tender.-Add ground beef and cook until browned.-Add tomatoes, tomato paste, parsley, basil, oregano, salt and pepper.-Simmer for 30 minutes."
1,,Spaghetti with tomato sauce,,"oil, pepper, tomato, clove, spaghetti, parsley, salt, cheese, basil, wine, juice, onion","-In a large pot of boiling salted water, cook the spaghetti until al dente.-Drain and return to the pot.-Meanwhile, in a large, deep skillet, heat the olive oil.-Add the onion and cook over moderate heat, stirring occasionally, until softened, about 5 minutes.-Add the garlic and cook, stirring, until fragrant, about 1 minute.-Add the tomatoes, wine, lemon juice, basil, salt and pepper and bring to a boil.-Simmer over moderate heat, stirring occasionally, until the sauce is thickened, about 20 minutes.-Add the spaghetti and toss well.-Add the parmesan and toss well.-Serve right away."
2,,Garlicky sauteed kale,,"oil, kale, pepper, salt, clove, spinach","-Heat olive oil in a large skillet over medium heat.-Add garlic and cook until fragrant, about 30 seconds.-Add kale and cook until wilted, about 2 minutes.-Season with salt and pepper."
3,,Sauteed spinach with garlic,,"oil, spinach, clove, salt, soy_sauce, pepper","-Heat oil in a large skillet over medium-high heat.-Add garlic; saute 30 seconds.-Add spinach; saute 1 minute.-Add soy sauce, salt, and pepper; toss to coat."
4,,Vanilla cupcakes,,"sugar, flour, butter, salt, egg, baking_powder, milk, extract, vanilla","-Preheat oven to 350 degrees.-Line muffin tins with paper liners.-In a large bowl, cream butter and sugar until light and fluffy.-Add eggs, one at a time, beating well after each addition.-Add vanilla and milk.-In a separate bowl, combine flour, baking powder and salt.-Add dry ingredients to creamed mixture alternately with milk, beginning and ending with dry ingredients.-Fill muffin cups 2/3 full.-Bake for 20-25 minutes or until a toothpick inserted in the center comes out clean.-Cool in pan for 5 minutes before removing to a wire rack to cool completely."
5,,Sugar cookies,,"sugar, flour, egg, salt, butter, baking_powder, vanilla, extract, baking_soda",-Preheat oven to 350 degrees.-Cream butter and sugar.-Add eggs and vanilla.-Mix well.-Add dry ingredients and mix well.-Roll dough into 1 inch balls and place on ungreased cookie sheet.-Flatten with bottom of glass dipped in sugar.-Bake for 8-10 minutes.
6,,Pizza dough,,"salt, flour, water, yeast, cheese, tomato, oil",-Mix yeast and warm water in a large bowl.-Let stand for 5 minutes.-Add flour and salt.-Mix well.-Knead on a floured surface for 5 minutes.-Place in a greased bowl and cover with a towel.-Let rise in a warm place for 1 hour.-Punch down and divide into 12 pieces.-Roll each piece into a ball.-Place on a greased baking sheet.-Cover and let rise for 1 hour.-Preheat oven to 450 degrees.-Brush with olive oil.-Bake for 10 minutes.-Remove from oven and sprinkle with cheese.-Bake for another 5 minutes.
7,,Pizza dough,,"salt, flour, water, yeast, sugar",-Mix all ingredients together.-Knead for 5 minutes.-Let rise for 30 minutes.-Roll out and cut into desired shapes.-Bake at 400 for 15 minutes.
8,,Banana smoothie,,"banana, milk, juice, ice",-Place all ingredients in a blender and blend until smooth.-Pour into 2 tall glasses and enjoy !
9,,Strawberry smoothie,,"strawberries, yogurt, ice, juice",-Place all ingredients in blender.-Blend until smooth.


# Pretrained ResNet101

In this section we will be showing how you can use a pre-trained network (trained on imagenet dataset), create much smaller embeddings and take advantage of Facebook's ingredients and instructions vocabularies.

You will see that results are comparable, even though we are working with much smaller embeddings.

As before, we will start by importing necessary packages.

In [43]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow import keras

In [44]:
import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

We will now create a custom model by taking the ResNet model and removing the last layer. This network is pre-trained on imagenet dataset.

In [ ]:
resnet_model = ResNet101(include_top=True, 
                         weights='imagenet', 
                         input_tensor=None, 
                         input_shape=None, 
                         pooling=None, 
                         classes=1000)

In [46]:
cust_model = keras.Model(resnet_model.input, resnet_model.layers[-2].output)

We will now create embeddings using this slightly modified network.

In [47]:
if create_vectors:
    items_to_upload_cust = []

    i=1
    for img_file in demo_files:
        print(f'{i}/{len(demo_files)}', end='\r')
        i = i+1

        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        features = cust_model.predict(x)

        items_to_upload_cust.append((f'{img_file}', features[0]))

To avoid waiting for vectors to be created, we are going to save them.

In [48]:
if create_vectors:
    with open('vectors_cust.pkl', 'wb') as f:
        pickle.dump(items_to_upload_cust, f)

In [49]:
if not create_vectors:
    items_to_upload_cust = pickle.load(open('vectors_cust.pkl', 'rb'))

We will now create a new index and upsert vectors.

In [50]:
index_name_cust = 'recipe-search-custom'

In [ ]:
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name_cust)

In [ ]:
pinecone.create_index(name=index_name_cust, metric='cosine', shards=1)

In [ ]:
index_cust = pinecone.Index(name=index_name_cust)
index_cust.info()

## Upsert

In [54]:
for i in range(0, len(items_to_upload_cust), 1000): 
    acks = index_cust.upsert(items=items_to_upload_cust[i:i+1000],
                             disable_progress_bar=True)

In [55]:
index_cust.info()

InfoResult(index_size=10000, dimension=0)

## Query

Finally, we will use the same test images to check the performance of our custom model. 

In [59]:
from inversecooking.src.utils.output_utils import prepare_output
from tensorflow.keras.preprocessing import image
import urllib
from PIL import Image


titles_display_cust = []
ingredients_display_cust = []
instructions_display_cust = []
images_display_cust = []

thresh_score = 0.4

for img_file in demo_urls:
    response = requests.get(img_file)
    img = Image.open(BytesIO(response.content))
    
    transf_list = []
    transf_list.append(transforms.Resize(256))
    transf_list.append(transforms.CenterCrop(224))
    transform = transforms.Compose(transf_list)

    image_transf = transform(img)

    x = image.img_to_array(image_transf)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = cust_model.predict(x)[0]
    
    query_results = index_cust.query(queries=[features], top_k=2, disable_progress_bar=True)
    
    for img_file, score in zip(query_results[0].ids, query_results[0].scores):
        if score > thresh_score:
          
            images_display_cust.append(img_file)

            outs, valid = prepare_output(instructions[img_file],
                                         ingredients[img_file], 
                                         ingrs_vocab, 
                                         vocab)

            if not valid:
                print ("Not a valid recipe!")
                print ("Reason: ", valid['reason'])
                continue
            else:
                titles_display_cust.append(outs['title'])
                ingredients_display_cust.append(', '.join(outs['ingrs']))
                instructions_display_cust.append('-'+'\n-'.join(outs['recipe']))


        else:
            print('No recipes found.')
            break

## Display results

In [91]:
# Results
instructions_display_cust = [instruction.replace('\n', '<br>') for instruction in instructions_display_cust]

df_r = pd.DataFrame({'query_image': [entry for entry in demo_urls for _ in range(2)],
                     'similar recipe': titles_display_cust, 
                     'recipe_image': images_display_cust,
                     'ingredients': ingredients_display_cust, 
                     'instructions': instructions_display_cust})

# Show results
display_table(df_r, titles=['Results'])

,query_image,similar recipe,recipe_image,ingredients,instructions
0,,Pasta with tomato sauce,,"oil, pepper, tomato, clove, onion, salt, pasta, parsley, basil","-Heat the olive oil in a large skillet over medium heat.-Add the onion and garlic and cook until the onion is translucent, about 5 minutes.-Add the tomatoes, tomato paste, salt, pepper, and basil.-Bring to a boil, then reduce the heat to low and simmer for about 20 minutes.-Meanwhile, cook the pasta according to the package directions.-Drain the pasta and add it to the tomato sauce.-Toss to coat.-Serve immediately."
1,,Parmesan omelet,,"pepper, oil, egg, butter, cheese, tomato, salt","-In a small bowl, beat eggs with salt and pepper.-In a large skillet, heat oil over medium heat.-Add eggs and cook, stirring constantly, until eggs are set.-Remove from heat.-Add cheese and tomatoes to eggs.-Fold in omelet and serve."
2,,Komatsuna namul,,"seeds, soy_sauce, salt, sugar, oil","-Parboil the komatsuna in salted water, then squeeze out excess water.-Cut into 5 cm lengths.-Combine the ingredients in a bowl, and mix well.-Add the komatsuna and toss."
3,,Mongolian beef,,"soy_sauce, oil, sugar, steak, ginger, clove, onion, pepper","-In a small bowl, combine soy sauce, brown sugar, sesame oil, garlic, ginger and red pepper flakes.-Place steak in a large resealable plastic bag.-Pour soy sauce mixture over steak, seal bag, and turn to coat.-Marinate in the refrigerator for at least 1 hour, turning occasionally.-Preheat an outdoor grill for medium-high heat and lightly oil grate.-Remove steak from marinade, and discard marinade.-Grill steak on preheated grill until desired doneness, about 5 minutes per side for medium."
4,,Vanilla cupcakes,,"sugar, flour, butter, salt, egg, baking_powder, milk, extract, vanilla","-Preheat oven to 350 degrees.-Line muffin tins with paper liners.-In a large bowl, cream butter and sugar until light and fluffy.-Add eggs, one at a time, beating well after each addition.-Add vanilla and milk.-In a separate bowl, combine flour, baking powder and salt.-Add dry ingredients to creamed mixture alternately with milk, beginning and ending with dry ingredients.-Fill muffin cups 2/3 full.-Bake for 20-25 minutes or until a toothpick inserted in the center comes out clean.-Cool in pan for 5 minutes before removing to a wire rack to cool completely."
5,,Candy cane sugar,,"sugar, food_coloring, water, candies","-In a large bowl, combine the confectioners' sugar, water and peppermint candies.-Mix until well blended.-Tint with food coloring if desired."
6,,Pizza dough,,"cheese, tomato, pepper, pizza_dough, salt, yeast, oil, flour, water","-In a large bowl, dissolve yeast in warm water.-Let stand until creamy, about 10 minutes.-Stir in flour, salt, and oil.-Knead dough on a lightly floured surface until smooth and elastic, about 8 minutes.-Place in a greased bowl, turning once to grease top.-Cover and let rise in a warm place until doubled, about 1 hour.-Punch down dough and divide into 12 pieces.-Roll each piece into a 12-inch circle.-Place on a greased baking sheet.-Cover and let rise until doubled, about 1 hour.-Preheat oven to 450 degrees f (230 degrees c).-Bake in preheated oven until golden brown, about 10 minutes."
7,,Baked ziti,,"cheese, pepper, onion, oil, tomato, salt, basil, clove","-Preheat oven to 350 degrees f.-In a large bowl, combine the tomatoes, onion, garlic, basil, salt, pepper, and olive oil.-Mix well.-In a 9x13 baking dish, layer half of the ziti, half of the tomato mixture, half of the mozzarella, and half of the parmesan.-Repeat layers.-Bake for 30 minutes."
8,,Berry banana smoothie,,"yogurt, strawberries, banana, milk, honey, raspberries",-Place all ingredients in a blender and blend until smooth.-Pour into glasses and serve.
9,,Strawberry banana smoothie,,"strawberries, milk, yogurt, ice, banana",-Place all ingredients in a blender and blend until smooth.-Pour into glasses and enjoy !


Even though embeddings coming from Facebook's model were slightly better, this pretrained model gave us pretty reasonable results and with some fine-tuning, it could work just as well.

The advantage of using smaller embeddings is the cost of storing these embeddings and the time needed to create them and insert them. The disadvantage is that - the smaller the embeddings, the less information they hold.

## Delete the index

Delete the index once you are sure that you do not want to use it anymore. Once it is deleted, you cannot reuse it.

In [ ]:
%%script echo skipping
pinecone.delete_index(index_name)
pinecone.delete_index(index_name_cust)